In [1]:
import pandas as pd
import dash
from dash import html, dcc, Input, Output

# Load data
data = pd.read_csv(r'C:\Users\sanba\Desktop\Me\cmc\PROJET\PFE\MODEL\data_final_nodeling.csv')

# Clean data
data['TYPE'] = data['TYPE'].fillna('').astype(str)
data['MARQUE'] = data['MARQUE'].fillna('').astype(str)

# Unique types
types = sorted([t for t in data['TYPE'].unique() if t])

# Last row fallback
last_row = data.iloc[-1]

# Icons
icon_map = {
    'Mise en circulation': '/assets/icon_date.png',
    'Date visite': '/assets/date_visite.png',
    'TYPE': '/assets/type.png',
    'MARQUE': '/assets/Marque.png',
    'POIDS': '/assets/Poids.png',
    'STATUS': '/assets/STATUS.png',
    'EFFICACITE': '/assets/efficacite.png'
}

# App
app = dash.Dash(__name__)

# Layout
app.layout = html.Div(
    style={
        'backgroundImage': 'linear-gradient(to top, #f8e850, #e6f0ff)',
        'height': '100vh',
        'padding': '20px',
        'fontFamily': 'Arial, sans-serif',
        'position': 'relative'
    },
    children=[
        html.Img(
            src='/assets/logo2.png',
            style={
                'position': 'absolute',
                'right': '390px',
                'top': '6%',
                'transform': 'translateY(-55%)',
                'height': '500px',
            }
        ),

        html.Div(
            style={
                'color': '#001F8A',
                'background': 'linear-gradient(to top, #f8e850, #e6f0ff)',
                'padding': '15px 25px',
                'borderRadius': '36px',
                'textAlign': 'center',
                'width': 'fit-content',
                'border': '2px solid #001F8A',
                'boxShadow': '0px 4px 8px rgba(0, 0, 0, 0.1)',
                'position': 'absolute',
                'left': '30px',
                'top': '16%',
                'display': 'flex',
                'alignItems': 'center',
                'gap': '12px',
                'minWidth': '180px',
                'flexDirection': 'column',
                'fontWeight': '900',
                'fontSize': '20px',
            },
            children=[
                html.Label('Type:', style={'color': '#001F8A', 'marginBottom': '8px'}),
                dcc.Dropdown(
                    id='type-dropdown',
                    options=[{'label': t, 'value': t} for t in types],
                    placeholder="Sélectionnez",
                    clearable=True,
                    style={
                        'width': '130px',
                        'color': '#001F8A',
                        'borderRadius': '12px',
                        'border': '1.5px solid #001F8A',
                    }
                )
            ]
        ),

        html.Img(
            src='/assets/Car.png',
            style={
                'position': 'absolute',
                'top': '55%',
                'right': '9px',
                'transform': 'translateY(-50%)',
                'height': '500px',
            }
        ),

        html.Div(id='info-div')
    ]
)


@app.callback(
    Output('info-div', 'children'),
    Input('type-dropdown', 'value')
)
def update_info(selected_type):
    if selected_type:
        filtered = data[data['TYPE'] == selected_type].iloc[-1]
    else:
        filtered = last_row

    props = [
        ('Mise en circulation', filtered.get('MIS EN CIRCULATION', ''), '800px', '2%'),
        ('Date visite', filtered.get('DATE VISITE', ''), '552px', '2%'),
        ('TYPE', filtered.get('TYPE', ''), '30px', '36%'),
        ('MARQUE', filtered.get('MARQUE', ''), '30px', '51%'),
        ('POIDS', filtered.get('POIDS', ''), '30px', '66%'),
        ('STATUS', filtered.get('status', ''), '30px', '81%'),
        ('EFFICACITE', filtered.get('EFFICACITE', ''), '848px', '51%'),
    ]

    boxes = []
    for title, value, left, top in props:
        icon_src = icon_map.get(title, '/assets/logo.png')
        box = html.Div(
            style={
                'color': '#001F8A',
                'background': 'linear-gradient(to top, #f8e850, #e6f0ff)',
                'padding': '15px 25px',
                'borderRadius': '36px',
                'textAlign': 'center',
                'width': 'fit-content',
                'border': '2px solid #001F8A',
                'boxShadow': '0px 4px 8px rgba(0, 0, 0, 0.1)',
                'position': 'absolute',
                'left': left,
                'top': top,
                'display': 'flex',
                'alignItems': 'center',
                'gap': '12px',
                'minWidth': '180px',
            },
            children=[
                html.Img(src=icon_src, style={'height': '50px', 'width': '50px'}),
                html.Div([
                    html.H2(title, style={
                        'margin': 0,
                        'paddingBottom': '5px',
                        'fontSize': '16px',
                        'fontWeight': '900',
                        'color': '#001F8A',
                    }),
                    html.Div(str(value), style={
                        'fontWeight': '900',
                        'fontSize': '14px',
                        'color': '#001F8A',
                    }),
                ])
            ]
        )
        boxes.append(box)

    # ✅ Add effectiveness per wheel
    try:
        poids = float(filtered['POIDS']) if filtered['POIDS'] else 1
        efficacite_globale = float(filtered['EFFICACITE']) if filtered['EFFICACITE'] else 0

        effs = {
            'FORCE_AV_G': (float(filtered['FORCE_AV_G']) * efficacite_globale) / poids,
            'FORCE_AV_D': (float(filtered['FORCE_AV_D']) * efficacite_globale) / poids,
            'FORCE_AR_G': (float(filtered['FORCE_AR_G']) * efficacite_globale) / poids,
            'FORCE_AR_D': (float(filtered['FORCE_AR_D']) * efficacite_globale) / poids
        }

        def get_color(eff):
            return 'red' if eff < 12 else 'green'

        force_boxes = [
            ('FORCE_AV_G', filtered['FORCE_AV_G'], effs['FORCE_AV_G'], '300px', '25%'),
            ('FORCE_AV_D', filtered['FORCE_AV_D'], effs['FORCE_AV_D'], '810px', '25%'),
            ('FORCE_AR_G', filtered['FORCE_AR_G'], effs['FORCE_AR_G'], '300px', '78%'),
            ('FORCE_AR_D', filtered['FORCE_AR_D'], effs['FORCE_AR_D'], '810px', '78%')
        ]

        for title, force_value, eff_value, left, top in force_boxes:
            color = get_color(eff_value)
            box = html.Div(
                style={
                    'color': color,
                    'background': 'transparent',
                    'padding': '10px 15px',
                    'borderRadius': '36px',
                    'textAlign': 'center',
                    'width': 'fit-content',
                    'border': f'2px solid {color}',
                    'position': 'absolute',
                    'left': left,
                    'top': top,
                    'fontWeight': '900',
                    'fontSize': '16px',
                    'boxShadow': f'0px 0px 10px {color}',
                    'minWidth': '140px'
                },
                children=[
                    html.Div([
    html.P(f"{force_value}", style={'marginBottom': '2px', 'color': color}),
    html.P(f" {eff_value:.1f}%", style={'marginTop': '0', 'color': color})
])
                ]
            )
            boxes.append(box)

    except Exception as e:
        print(f"Erreur calcul efficacité: {e}")

    return boxes


if __name__ == '__main__':
     app.run(debug=True)
